Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 154.59423950 0.263821102 0.0041713779 0.0040688455 1000
 beta   0.90638023 0.041269033 0.0006525207 0.0006892418 1000
sigma   5.09978729 0.189980921 0.0030038621 0.0030937633 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0679500 154.4170000 154.597000 154.76900000 155.1130250
 beta   0.8254523   0.8791735   0.907081   0.93428475   0.9865398
sigma   4.7542623   4.9681800   5.094930   5.22545250   5.4764555



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.441 0.978855 5.01834; 155.112 0.830961 5.14387; … ; 154.685 0.905007 5.09064; 154.619 0.88024 5.26477]

Union{Missing, Float64}[154.491 0.898241 4.71288; 154.809 0.908826 5.23233; … ; 154.281 0.913903 5.28347; 154.758 0.917595 5.22389]

Union{Missing, Float64}[155.178 0.860376 5.16057; 154.164 0.983547 5.01183; … ; 154.411 0.915011 5.11366; 154.361 0.897381 5.07476]

Union{Missing, Float64}[154.356 0.944227 4.97141; 154.826 0.863457 5.21682; … ; 154.629 0.952278 5.02912; 154.67 0.895017 4.95821]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean          SD       Naive SE       MCSE      ESS
alpha 154.59423950 0.263821102 0.0041713779 0.0040688455 1000
 beta   0.90638023 0.041269033 0.0006525207 0.0006892418 1000
sigma   5.09978729 0.189980921 0.0030038621 0.0030937633 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%       97.5%   
alpha 154.0679500 154.4170000 154.597000 154.76900000 155.1130250
 beta   0.8254523   0.8791735   0.907081   0.93428475   0.9865398
sigma   4.7542623   4.9681800   5.094930   5.22545250   5.4764555



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*